## Preface

The purpose of this script is to match EMBL gene IDs with those from RefSeq. We will use the peptide library from EMBL as the query database and the RefSeq as the subject. From there we will map the BLAST hits to Orthogroups.

The working directory is the jobs folder.

In [1]:
import pandas as pd
import Bio.SeqIO as SeqIO
from Bio import SearchIO
import os as os

## FASTA Input

Here we are loading in the Medicago Truncatula (v4.0) sequences from EMBL.

In [2]:
#read in fasta files
embl = SeqIO.to_dict(SeqIO.parse("../raw_data/Medicago_truncatula.MedtrA17_4.0.pep.all.fa", "fasta"))

#extract ids
embl_ids = list(embl.keys())

#extract sequences. doing this iterative search will keep both the ids and sequences in the same order for easier reference
embl_seq = []

for i in embl_ids:
    embl_seq.append(embl[i].seq)

## BLAST Alignment

BLAST Ensembl sequences against RefSeq versions. Save data to disk because it can be computationally intensive.

In [3]:
#construct BLAST database
os.system("makeblastdb -in ../20200324_genome_analyses/raw_data/protein_fasta/AM_refseq_medicago_truncatula.FAA -out ../blast_db/medicago_truncatula -dbtype prot")

0

In [4]:
"""
No need to re-run this because it takes forever and I did it already.
"""
#construct directory to store blast alignments
os.mkdir("../processed_data/Q-medicagoENSEMBL_S-medicagoREFSEQ")

#write out embl sequences to file to make life easier
os.mkdir("../raw_data/medicago_truncatula_embl_seq")

for record in SeqIO.parse("../raw_data/Medicago_truncatula.MedtrA17_4.0.pep.all.fa", "fasta"):
    SeqIO.write(record, "../raw_data/medicago_truncatula_embl_seq/" + record.id + ".FAA", "fasta")

seq_files = os.listdir("../raw_data/medicago_truncatula_embl_seq")

In [5]:
print(len(seq_files))
print(len(embl_ids))
print(len(set(embl_ids)))
print(seq_files[0])

57585
57585
57585
AES58618.FAA


In [6]:
"""
I already ran this. No need to run it again.
"""
#run blast alignments and save to disk
db = "../blast_db/medicago_truncatula"
for i in seq_files:
    cmd = "blastp -query ../raw_data/medicago_truncatula_embl_seq/" + i + " -db " + db + " -outfmt 7 -out ../processed_data/Q-medicagoENSEMBL_S-medicagoREFSEQ/" + i + ".txt"
    os.system(cmd)

## Extracting BLAST Data

In [13]:
#iterate through blast files and save most relevant information
embl_gene = []
refseq_gene = []
ident_pcts = []
aln_spans = []
evalues = []

file_prefix = "../processed_data/Q-medicagoENSEMBL_S-medicagoREFSEQ/"
files = os.listdir(file_prefix)

for i in files:
    try:
        blast_in = SearchIO.read(file_prefix + i, "blast-tab", comments = True)
        embl_gene.append(blast_in[0][0].query_id)
        refseq_gene.append(blast_in[0][0].hit_id)
        ident_pcts.append(blast_in[0][0].ident_pct)
        aln_spans.append(blast_in[0][0].aln_span)
        evalues.append(blast_in[0][0].evalue)
    except:
        pass

In [14]:
#convert blast info lists into python dataframe
blast_df = pd.DataFrame(list(zip(embl_gene, refseq_gene, ident_pcts, aln_spans, evalues)),
                       columns = ["embl_gene", "refseq_gene", "percent_identity", "alignment_length", "evalue"])
blast_df.to_csv("../processed_data/20210706_medicago_blast_results.tsv", sep = "\t")

In [2]:
blast_df = pd.read_csv("../processed_data/20210706_medicago_blast_results.tsv", sep = "\t", index_col = 0)

#I'm choosing a relatively stringent requirement of e-value < 0.0001 as an accepted cut-off
blast_df["evalue_acceptance"] = blast_df.evalue < 0.0001

## Orthogroup Matching

In [4]:
#read in orthogroup data
ortho_meta = pd.read_csv("../20200324_genome_analyses/metadata/Orthogroups.csv", sep = "\t")

In [5]:
#create dataframe matching brachypodium proteins with orthogroup
ortho_filt = ortho_meta[["Unnamed: 0", "AM_refseq_medicago_truncatula"]]
ortho_filt = ortho_filt.dropna()

#match proteins to orthogroups
orthogroup = []
protein = []

for i in range(0,len(ortho_filt.index)):
    holder = ortho_filt.iloc[i,1]
    for j in holder.split(", "):
        orthogroup.append(ortho_filt.iloc[i,0])
        protein.append(j)

refseq_ortho = pd.DataFrame(list(zip(orthogroup, protein)),
                          columns = ["orthogroup", "refseq_gene"])

In [14]:
blast_df2 = pd.merge(blast_df, refseq_ortho, on = "refseq_gene")
blast_df2.loc[(blast_df2.evalue_acceptance == False), "orthogroup"] = "not_in_refseq"

#load ENA metadata for matching protein accessions to genes
ena = pd.read_csv("../metadata/Medicago_truncatula.MedtrA17_4.0.51.ena.tsv", sep = "\t")
blast_df2 = pd.merge(blast_df2, ena, left_on = "embl_gene", right_on = "protein_stable_id")

#replace MTR_ with Medtr in gene stable ids to match jcvi conventions
blast_df2["jcvi"] = blast_df2["gene_stable_id"].replace({"MTR_" : "Medtr"}, regex=True)

#This dataframe has multiple entries for each protein because ENA marks all the different kinds of accession in different rows for the same gene
blast_df2.to_csv("../processed_data/20210706_medicago_blast_with_orthogroup.tsv", sep = "\t")